## Tutorial on the one-class SVM

## 1. Configurations

In [1]:
from    typing      import  *
from    pathlib     import  Path

import  numpy   as  np

from    sklearn.svm     import  OneClassSVM
from    sklearn.metrics import  roc_auc_score, classification_report

from    utils   import  *
from    helper  import  *

In [2]:
# Set the root (Category: hazelnut)
base_path = Path(BASE_PATHS[MVTEC_AD])
category = 'hazelnut'
root = base_path/category

# Set the directories
train_normal_dir    = root / "train" / "good"
test_normal_dir     = root / "test" / "good"
test_anomaly_dirs   = [dir_subcat for dir_subcat in (root/"test").glob('*') if dir_subcat.name != "good"]

## 2. Load data

In [3]:
# Normal train/test data
X_train = load_images(train_normal_dir)
X_test  = load_images(test_normal_dir)
y_test  = np.zeros(len(X_test))  # Normal: 0

# Anomalous test data
for anomaly_dir in test_anomaly_dirs:
    X_anom = load_images(anomaly_dir)
    X_test = np.vstack([X_test, X_anom])
    y_test = np.hstack([y_test, np.ones(len(X_anom))])  # Anomalous: 1

print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Train data shape: (391, 16384)
Test data shape: (110, 16384) (110,)


## 3. Train the model

In [4]:
ocsvm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.01)
ocsvm.fit(X_train)

,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,tol,0.001
,nu,0.01
,shrinking,True
,cache_size,200
,verbose,False
,max_iter,-1


## 4. Test the model

In [5]:
# Decision_function
# * Positive value indicates that the sample is predicted as an inlier
# * Negative value indicates that the sample is predicted as an outlier
scores = -ocsvm.decision_function(X_test)
"""Higher value indicates higher probability of being predicted as an outlier (`1`); lower value indicates higher probability of being predicted as an inlier (`0`)."""
y_pred = ocsvm.predict(X_test)

# ROC AUC without threshold
auc = roc_auc_score(y_test, scores)
print(f"ROC-AUC: {auc:.4f}")

# Setting the threshold (Example: 0)
y_pred = ocsvm.predict(X_test)  # 1 for inliers, -1 for outliers
y_pred = np.where(y_pred==-1, 1, 0)  # -1 -> anomaly (1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

ROC-AUC: 0.7211

Classification Report:
              precision    recall  f1-score   support

         0.0     0.4545    1.0000    0.6250        40
         1.0     1.0000    0.3143    0.4783        70

    accuracy                         0.5636       110
   macro avg     0.7273    0.6571    0.5516       110
weighted avg     0.8017    0.5636    0.5316       110



End of file